In [1]:
import bioread
import pandas as pd
from pyPPG import PPG, Fiducials, Biomarkers
from pyPPG.datahandling import load_data, plot_fiducials, save_data
import pyPPG.preproc as PP
import pyPPG.fiducials as FP
import pyPPG.biomarkers as BM
import pyPPG.ppg_sqi as SQI
import pyPPG
import matplotlib.pyplot as plt
import numpy as np
import sys
import json
import pandas as pd
import os

In [2]:
# read csv file
metadata = pd.read_csv('metadata.csv')

In [3]:
sugar = metadata['blood glucose']
age =  metadata['age']
height = metadata['height']
weight = metadata['weight']
ids = metadata['person_id']

In [4]:
ppg_data = []
for i in range(0,  len(ids)):
    filepath = "/Users/sahilsahu/Desktop/folders/Feature-Extraction/code/data/exp3/ppg/acq"
    filepath += "/P" + str(ids[i]) + ".acq"
    
    # if(os.path.exists(filepath) == False and not np.isnan(sugar[i])):
    #     print("File not found: ", names[i])
    if not np.isnan(sugar[i]):
        acq_data =  bioread.read_file(filepath)
        ndata = {
                "name" : "P" + str(ids[i]),
                "sugar" : sugar[i],
                "age" : age[i],
                "height" : height[i],
                "weight" : weight[i],
                "ppg_raw" : acq_data.channels[0].data,
                "ppg_signal" : None
                
        }
        ppg_data.append(ndata)

In [5]:
for i in range(0, len(ppg_data)):
    csv_path = "/Users/sahilsahu/Desktop/folders/Feature-Extraction/code/data/exp3/ppg/csv"
    csv_path += "/" + ppg_data[i]['name'] + ".csv"
    ppg_data[i]['ppg_signal'] = load_data(data_path=csv_path, start_sig=1, fs=2000, end_sig=-1)

In [6]:
signals_list = []
for i in range(0, len(ppg_data)):
    signals_list.append(ppg_data[i]['ppg_signal'])

In [7]:
for signal in signals_list:
    prep = PP.Preprocess(fL=signal.fL, fH=signal.fH, order=signal.order, sm_wins=signal.sm_wins)
    signal.ppg, signal.vpg, signal.apg, signal.jpg = prep.get_signals(s=signal)


In [8]:
for signal in signals_list:
    corr_on = ['on', 'dn', 'dp', 'v', 'w', 'f']
    correction=pd.DataFrame()
    correction.loc[0, corr_on] = True
    signal.correction=correction


signals_list_ppg_class = []

for signal in signals_list:
    signals_list_ppg_class.append(PPG(signal))

In [9]:
ficudals_list = []

for signal in signals_list_ppg_class:
    fpex = FP.FpCollection(signal)
    fiducials = fpex.get_fiducials(signal)
    fp = Fiducials(fiducials)
    ppgSQI = round(np.mean(SQI.get_ppgSQI(signal.v, signal.fs, fp.sp)) * 100, 2)
    print('Mean PPG SQI: ', ppgSQI, '%')
    ficudals_list.append(fp)

Mean PPG SQI:  97.9 %
Mean PPG SQI:  93.53 %
Mean PPG SQI:  94.72 %
Mean PPG SQI:  97.38 %
Mean PPG SQI:  93.53 %
Mean PPG SQI:  97.55 %
Mean PPG SQI:  97.61 %
Mean PPG SQI:  97.38 %
Mean PPG SQI:  98.58 %
Mean PPG SQI:  93.25 %
Mean PPG SQI:  97.02 %
Mean PPG SQI:  96.96 %
Mean PPG SQI:  95.95 %
Mean PPG SQI:  96.78 %
Mean PPG SQI:  98.16 %
Mean PPG SQI:  96.98 %
Mean PPG SQI:  96.24 %
Mean PPG SQI:  96.68 %
Mean PPG SQI:  94.35 %
Mean PPG SQI:  93.26 %
Mean PPG SQI:  97.21 %
Mean PPG SQI:  98.33 %
Mean PPG SQI:  97.47 %
Mean PPG SQI:  98.43 %
Mean PPG SQI:  97.55 %
Mean PPG SQI:  94.64 %
Mean PPG SQI:  93.75 %
Mean PPG SQI:  94.0 %
Mean PPG SQI:  98.52 %
Mean PPG SQI:  92.92 %
Mean PPG SQI:  92.59 %
Mean PPG SQI:  98.1 %
Mean PPG SQI:  98.63 %
Mean PPG SQI:  95.75 %
Mean PPG SQI:  94.14 %
Mean PPG SQI:  91.14 %
Mean PPG SQI:  97.08 %
Mean PPG SQI:  96.64 %
Mean PPG SQI:  89.59 %
Mean PPG SQI:  95.45 %
Mean PPG SQI:  95.52 %
Mean PPG SQI:  90.53 %
Mean PPG SQI:  97.09 %
Mean PPG SQI: 

In [10]:
biomarkers_list = []
for i in range(len(signals_list_ppg_class)):
    s = signals_list_ppg_class[i]
    fp = ficudals_list[i]
    # Init the biomarkers package
    bmex = BM.BmCollection(s=s, fp=fp)
    # Extract biomarkers
    bm_defs, bm_vals, bm_stats = bmex.get_biomarkers()
    tmp_keys=bm_stats.keys()
    print('Statistics of the biomarkers:')
    for i in tmp_keys: print(i,'\n',bm_stats[i])

    # Create a biomarkers class
    bm = Biomarkers(bm_defs=bm_defs, bm_vals=bm_vals, bm_stats=bm_stats)
    
    biomarkers_list.append(bm)
    

no more peaks
no more peaks
no more peaks
no more peaks
Statistics of the biomarkers:
ppg_sig 
                     Tpi       Tpp      Tsys      Tdia        Tsp       Tdp  \
Statistics                                                                   
mean           0.761934  0.761187  0.383801  0.376139   0.161747  0.385794   
median         0.761500  0.761750  0.383000  0.379000   0.161500  0.384750   
std            0.030023  0.028847  0.020222  0.034493   0.012363  0.020201   
percentile_25  0.738625  0.741625  0.370625  0.352500   0.156000  0.372125   
percentile_75  0.783000  0.782625  0.396250  0.401000   0.167375  0.398250   
iqr            0.044375  0.041000  0.025625  0.048500   0.011375  0.026125   
skew          -0.123669 -0.169147  0.624486 -0.033513   4.271099  0.593837   
kurtosis      -0.358012  0.146453  1.654978 -0.088591  37.941950  1.563891   
mad            0.024509  0.023176  0.015692  0.028037   0.007430  0.015750   

                 deltaT     Tsw10     Tsw25  

In [11]:

# Initialize an empty DataFrame for the final result
final_df = None
i = 0
# Iterate over each Biomarkers instance in biomarkers_list
for biomarker in biomarkers_list:
    # Extract biomarker data using the get_bm method
    biomarkers_category_keys = ['ppg_sig', 'sig_ratios']
    biomarker_df = pd.DataFrame()
    biomarker_dic = {}
    
    for biomarkers_category_key in biomarkers_category_keys:
        ppg_sig_stats = biomarker.bm_stats[biomarkers_category_key]
        biomarker_keys = ppg_sig_stats.keys()
        
        for biomarker_key in biomarker_keys:
            statistics_keys = ppg_sig_stats[biomarker_key].keys()
            for statistics_key in statistics_keys:
                col = biomarker_key + '_' + statistics_key
                val = ppg_sig_stats[biomarker_key][statistics_key]
                biomarker_dic[col] = val
    biomarker_df = pd.DataFrame(biomarker_dic, index=[i])
    final_df = pd.concat([final_df, biomarker_df])
        
        
    

In [12]:
age_list = []
height_list = []
weight_list = []
sugar_list = []
diabetes_list = []
name_list = []
diabetes = metadata['diabetes']

for i in range(0, len(ppg_data)):
    age_list.append(ppg_data[i]['age'])
    height_list.append(ppg_data[i]['height'])
    weight_list.append(ppg_data[i]['weight'])
    sugar_list.append(ppg_data[i]['sugar'])
    name_list.append(ppg_data[i]['name'])
    
    for j in range(0, len(ids)):
        if "P" + str(ids[j]) == str(ppg_data[i]['name']):
            print(diabetes[j])
            if(diabetes[j] == 'yes'):
                diabetes_list.append(1)
            else:
                diabetes_list.append(0)
            break


no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
yes
no
no
no
yes
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no 
no
no
no
no
no
no
yes


In [13]:
final_df['age'] = age_list
final_df['height'] = height_list
final_df['weight'] = weight_list
final_df['diabetes'] = diabetes_list
final_df['sugar'] = sugar_list

In [14]:
final_df

,Tpi_mean,Tpi_median,Tpi_std,Tpi_percentile_25,Tpi_percentile_75,Tpi_iqr,Tpi_skew,Tpi_kurtosis,Tpi_mad,Tpp_mean,...,Asp/(Tpi-Tsp)_percentile_75,Asp/(Tpi-Tsp)_iqr,Asp/(Tpi-Tsp)_skew,Asp/(Tpi-Tsp)_kurtosis,Asp/(Tpi-Tsp)_mad,age,height,weight,diabetes,sugar
0,0.761934,0.76150,0.030023,0.738625,0.783000,0.044375,-0.123669,-0.358012,0.024509,0.761187,...,0.918094,0.065496,0.935396,1.750709,0.042687,38,165,59.3,0,117.24
0,0.775829,0.77350,0.059216,0.732750,0.817250,0.084500,0.024998,-0.228823,0.048021,0.774923,...,1.561568,0.229315,0.061885,-0.006827,0.134129,42,163,49.4,0,88.19
0,0.735244,0.74700,0.051358,0.705375,0.767250,0.061875,-0.617952,0.061125,0.040331,0.735061,...,0.593492,0.073333,0.016122,0.662910,0.045611,36,171,99.3,0,134.05
0,0.767885,0.76500,0.040333,0.740500,0.793000,0.052500,2.104113,12.705569,0.029768,0.765494,...,1.160484,0.251324,0.051974,-1.035210,0.128208,26,165,60.0,0,100.31
0,0.710685,0.70900,0.074379,0.670250,0.747000,0.076750,4.142105,36.662932,0.047204,0.710779,...,1.624034,0.370440,1.154624,0.991969,0.279618,29,167,63.0,0,79.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.765616,0.75950,0.056987,0.729500,0.801500,0.072000,0.911800,1.697134,0.043979,0.763653,...,1.661740,0.304987,-0.094147,-0.172227,0.171815,27,163,59.0,0,84.84
0,0.617265,0.61725,0.045603,0.593375,0.645500,0.052125,-0.267723,1.007770,0.034796,0.617832,...,1.275097,0.227518,-1.101245,0.755068,0.166185,22,163,53.0,0,81.27
0,0.854344,0.84200,0.134804,0.811500,0.870500,0.059000,9.739692,105.402103,0.047355,0.853865,...,1.051787,0.221674,-1.942207,3.499490,0.175751,38,159,70.0,0,112.42
0,0.819036,0.82000,0.061828,0.781000,0.861500,0.080500,-0.180758,0.067239,0.049197,0.819668,...,2.296756,0.459392,-0.961241,0.822256,0.295460,55,148,53.0,0,107.62


In [15]:
# save to csv
final_df.to_csv('data_all.csv', index=False)